![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite.png)
# Isochrone Generation Example
Isochrones are a type of _iso line_—a contour that connects points sharing the same value. While contour lines link points of equal elevation and isotherms connect equal temperatures, isochrones outline regions that are reachable within a specific amount of travel time. They're commonly used to answer questions like: 'How far can I drive from here in 15 minutes?' or 'Which neighborhoods are within a 10-minute walk of a school?'

In this example we will generate `10`, `20`, and `30` minute isochrones for all the fire stations in California in the bounding box between Bakersfield and Merced. Then we will left join the places Overture table to those isochrones in order to label them as being within 10/20/30/>30 minutes from a fire station. Finally, we will generate clusters with DBSCAN to determine if there are regions where a fire station might be useful

## Define Sedona Context

In [ ]:
from sedona.spark import *
from  wherobots.sql.st_functions import ST_Isochrones
import pyspark.sql.functions as f
import os

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)


## Initialize Data

Preparing the data involves creating two dataframes:
* All the Overture places within a region of California. These are the locations for which we want to determine fire station proximity
* The fire departments from the above dataset. These are the locations from which we will generate isochrones

In [ ]:
polygon = "POLYGON ((-120.805664 35.32633, -118.861084 35.32633, -118.861084 37.378888, -120.805664 37.378888, -120.805664 35.32633))"
ca_places_df = sedona.table("wherobots_open_data.overture_2025_01_22_0.places_place").where(f"ST_Intersects(geometry, ST_GeomFromText('{polygon}'))")\
    .repartition(sedona.sparkContext.defaultParallelism * 2) # Havasu will only hit a few files; lets make sure our dataframe has enough partitions to utilize our cluster well.
fires_df = ca_places_df.where(f"categories.primary = 'fire_department'")
fires_df.count()

## Generate the isochrones

Here we generate isochrones for `10`, `20`, and `30` minutes for all 1,857 fire stations in our dataframe.

There are 4 arguments for the `ST_Isochrones()` function:

1. `geometry`: The starting point from which we will generate the isochrones.
2. `time_limits`: An array of travel times in minutes; In our case `10`, `20`, and `30`.
3. `mobility_type`: Transportation; In our case, `car`.
4. `inbound`: Indicates if the geometry is a destination (true) or origin (false); in our case the points represent the `origin`.
5. `isolate_contours`: If true, creates concentric rings; if false, creates overlapping isochrones; we are creating overlapping isochrones.


You'll notice this dataframe is marked for caching. Caching allows the database to plan the join efficiently without calculating the isochrones twice.

In the result below, you can see that the isochrones are returned as a list of polygons with the order matching the time limits. 

In [ ]:
%%time
isochrones_df = fires_df.withColumn(
    "isochrones",
    ST_Isochrones(f.col("geometry"), 
                  f.array(f.lit(10), f.lit(20), f.lit(30)),
                  f.lit("car"), 
                  f.lit(False), 
                  f.lit(False))
).cache()

isochrones_df.select("id","names","isochrones").show()

#### Map the Results with Sedona Kepler

In [ ]:
SedonaKepler.create_map(isochrones_df.withColumn("geometry", f.col("isochrones")[0]),name="Isochrones")

## How far away is the closest Fire Station to each Overture Maps Foundation _Places_ record?
Here, we assign each location in the Overture Maps Places dataset to its corresponding fire station isochrone bucket by performing a left spatial join and identifying the smallest bucket value from the intersecting isochrones.

In [ ]:
%%time

risk_df = (
    ca_places_df.alias("ca")
    .join(isochrones_df.alias("isochrones"), ST_Intersects(isochrones_df.isochrones[2], "ca.geometry"), "left")
    .withColumn(
        "bucket",
        f.when(isochrones_df.isochrones.isNull(), f.lit(None))
        .when(ST_Intersects(isochrones_df.isochrones[0], "ca.geometry"), 10)
        .when(ST_Intersects(isochrones_df.isochrones[1], "ca.geometry"), 20)
        .when(ST_Intersects(isochrones_df.isochrones[2], "ca.geometry"), 30)
    )
    .groupBy(f.col("ca.id")) 
    .agg(*[f.first("ca." + field).alias(field) for field in ca_places_df.columns if field != "id"], f.min(f.col("bucket")).alias("bucket"))
    .cache()
)

risk_df.write.format("geoparquet").mode("overwrite").save(os.getenv("USER_S3_PATH") + "fireRiskExampleIsochrones")
isochrones_df.unpersist()

risk_df.select("names","geometry","bucket").show()

## Find Clusters of OMF Place *not* near Fire Stations
Finally, we use WherobotsDB's DBSCAN to find clusters of places that are not close to a fire station.

In [ ]:
%%time
from math import pi

R = 6371000
C = 2 * pi * R
METERS_PER_DEGREE = C / 360.0

def degreesFromMeters(distanceInMeters):
    return distanceInMeters / METERS_PER_DEGREE


clustered_high_risk_df = (
    risk_df
    .where(risk_df.bucket > 10)
    .withColumn("dbscan_result", ST_DBSCAN(risk_df.geometry, degreesFromMeters(1000), 50, False))
    .groupBy("dbscan_result.cluster")
    .agg(ST_ConvexHull(ST_Union_Aggr("geometry")).alias("geometry"))
    .where("cluster != -1") # dont show outliers
).cache()

clustered_high_risk_df.count()

#### Map the Results with Sedona Kepler

In [ ]:
SedonaKepler.create_map(clustered_high_risk_df, name="Clusters")

## ST_Isochrones Function Documentation

For detailed information on the `ST_Isochrones` function, refer to the official Wherobots documentation: [ST_Isochrones](https://docs.wherobots.com/latest/references/wherobotsdb/vector-data/Function/#st_isochrones)